In [31]:
# Notebook vorbereiten
from common.functions import Notebook_vorbereiten
ab_ag_raw, columns, uniprot_data = Notebook_vorbereiten()
#War Strg+Shift+P --> Preferences: Open settings --> jupyter.notebookFileRoot richtig für das Problem mit dem Working Directory?

In [32]:
# spalten entfernen und missing antigen name zeilen
ab_ag = ab_ag_raw[["pdb", "antigen_name", "antigen_species", "Hchain", "Lchain", "antigen_chain"]]

pdb_count=ab_ag['pdb'].value_counts()
print(pdb_count.max())
print(pdb_count)

print("Anzahl fehlender Werte pro Spalte:")
print(ab_ag.isnull().sum())

# 1. Kopie des DataFrames erstellen
ab_ag_antigennamecleaned = ab_ag.copy()

# 2. Fehlende Werte in der Kopie entfernen
ab_ag_antigennamecleaned.dropna(subset=['antigen_name'], inplace=True)

# 3. Überprüfung
print("Anzahl fehlender Werte nach dem Löschen:")
print(ab_ag_antigennamecleaned['antigen_name'].isnull().sum())
print("Form des neuen DataFrames:", ab_ag_antigennamecleaned.shape)

# 4. Duplikate entfernen in pdb => wir müssen erst schauen, bei welchen Antikörpersequenz identisch
# ab_ag_neu1 = ab_ag_neu.drop_duplicates(subset=['pdb'], keep='first')

16
pdb
6vmk    16
3whe    12
7wow    12
8jmm    12
8t7r    12
        ..
8g05     1
6bck     1
8pq2     1
8hgm     1
7tas     1
Name: count, Length: 2675, dtype: int64
Anzahl fehlender Werte pro Spalte:
pdb                  0
antigen_name        33
antigen_species     52
Hchain              42
Lchain             179
antigen_chain       33
dtype: int64
Anzahl fehlender Werte nach dem Löschen:
0
Form des neuen DataFrames: (5490, 6)


# Funktionen aus dem sabdab_downloader importieren
from common.sabdab_downloader import getchothpdb
from common.sabdab_downloader import getannotation
from common.sabdab_downloader import getsequence

#
fab_list = ab_ag_raw[ab_ag_raw["pdb"] == pdb].values.tolist()

pdb_entries = ab_ag["pdb"].unique()
for pdb in pdb_entries:
    try:
        getchothpdb(pdb, "data/chothia")
    except Exception as e:
        print(f"Could not download {pdb}: {e}")

In [ ]:
# Funktionen aus dem sabdab_downloader importieren
from common.sabdab_downloader import getchothpdb
from common.sabdab_downloader import getannotation
import os

# Erstelle eine Liste mit allen eindeutigen PDB-Einträgen aus ab_ag
pdb_entries = ab_ag["pdb"].unique()

# Lade die Chothia-Daten und Chothia-nummerierten Sequenzen für jeden PDB-Eintrag herunter und speichere sie im Ordner des entsprechenden PDB-Eintrags
for pdb in pdb_entries:
    # Zeilenweise Einträge für den aktuellen PDB-Eintrag extrahieren (für die Chothia-annotierten Sequenzen)
    fab_list = ab_ag_raw[ab_ag_raw["pdb"] == pdb].values.tolist()
    # Erstelle die Ordner für Chothia-Daten und Chothia-annotierte Sequenz für den PDB-Eintrag, falls er nicht existiert
    if not os.path.exists(f"data/sabdab_downloads/{pdb}/chothia_file"):
        os.makedirs(f"data/sabdab_downloads/{pdb}/chothia_file")
    if not os.path.exists(f"data/sabdab_downloads/{pdb}/chothia_sequence"):
        os.makedirs(f"data/sabdab_downloads/{pdb}/chothia_sequence")
    # Lade die Chothia-Daten und Chothia-nummerierten Sequenzen herunter oder gib eine Fehlermeldung für entsprechende PDB-Einträge aus
    try:
        getchothpdb(pdb, f"data/sabdab_downloads/{pdb}/chothia_file")
    except Exception as e:
        print(f"Could not download Chothia-File for {pdb}: {e}")
    try:
        getannotation(pdb, fab_list, f"data/sabdab_downloads/{pdb}/chothia_sequence")
    except Exception as e:
        print(f"Could not download Chothia-annotated sequence for {pdb}: {e}")

KeyboardInterrupt: 